In [11]:
%pip -q install geopy pandas geopandas folium python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [17]:
import os

%load_ext dotenv
%dotenv
api_key = os.getenv('GOOGLE_MAPS_KEY')

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [35]:
import pandas as pd
df = pd.read_csv('aau_members.csv')
df.head()

,institution,location,country,website,status
0,Abia State University,Abia State,Nigeria,http://absu.edu.ng/,Full Member
1,Abm University College,Gaborone,Botswana,https://abm.ac.bw,Associate Member
2,Abrar University,Mogadishu,Somalia,https://abrar.edu.so,Associate Member
3,Abubakar Tafawa Balewa University,Bauchi,Nigeria,http://www.atbu.edu.ng/,Full Member
4,Accra Institute Of Technology,Accra,Ghana,http://ait-edu.us/,Full Member


In [36]:
from geopy.geocoders import Nominatim
from geopy.geocoders import GeoNames
from geopy.geocoders import GoogleV3

from geopy.extra.rate_limiter import RateLimiter
from functools import partial
from sklearn.metrics import get_scorer_names
from tqdm import tqdm

tqdm.pandas()

# geolocator = Nominatim(user_agent="clir_geocoder")
# geolocator = GeoNames(username="wsgrah", timeout=5, user_agent="example")
geolocator = GoogleV3(api_key)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df['location'] = df['institution'].progress_apply(partial(geocode))

100%|██████████| 440/440 [07:20<00:00,  1.00s/it]


In [37]:
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df.head()

,institution,location,country,website,status,point
0,Abia State University,"(441103, Uturu, Abia, Nigeria, (5.8225266, 7.3...",Nigeria,http://absu.edu.ng/,Full Member,"(5.8225266, 7.3940761, 0.0)"
1,Abm University College,"(9VFM+H73, Lephala, Gaborone, Botswana, (-24.6...",Botswana,https://abm.ac.bw,Associate Member,"(-24.626106, 25.8831748, 0.0)"
2,Abrar University,None,Somalia,https://abrar.edu.so,Associate Member,None
3,Abubakar Tafawa Balewa University,"(Campus: Along Bauchi, Dass Road, 740272, Yelw...",Nigeria,http://www.atbu.edu.ng/,Full Member,"(10.280075, 9.79455, 0.0)"
4,Accra Institute Of Technology,"(34 Concam Cres, Accra, Ghana, (5.5818774, -0....",Ghana,http://ait-edu.us/,Full Member,"(5.5818774, -0.2141298, 0.0)"


In [38]:
import geopandas as gpd

df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df["geometry"] = gpd.points_from_xy(df.longitude, df.latitude)

df.head()

,institution,location,country,website,status,point,latitude,longitude,altitude,geometry
0,Abia State University,"(441103, Uturu, Abia, Nigeria, (5.8225266, 7.3...",Nigeria,http://absu.edu.ng/,Full Member,"(5.8225266, 7.3940761, 0.0)",5.822527,7.394076,0.0,POINT (7.39408 5.82253)
1,Abm University College,"(9VFM+H73, Lephala, Gaborone, Botswana, (-24.6...",Botswana,https://abm.ac.bw,Associate Member,"(-24.626106, 25.8831748, 0.0)",-24.626106,25.883175,0.0,POINT (25.88317 -24.62611)
2,Abrar University,None,Somalia,https://abrar.edu.so,Associate Member,None,NaN,NaN,NaN,POINT EMPTY
3,Abubakar Tafawa Balewa University,"(Campus: Along Bauchi, Dass Road, 740272, Yelw...",Nigeria,http://www.atbu.edu.ng/,Full Member,"(10.280075, 9.79455, 0.0)",10.280075,9.794550,0.0,POINT (9.79455 10.28008)
4,Accra Institute Of Technology,"(34 Concam Cres, Accra, Ghana, (5.5818774, -0....",Ghana,http://ait-edu.us/,Full Member,"(5.5818774, -0.2141298, 0.0)",5.581877,-0.214130,0.0,POINT (-0.21413 5.58188)


In [39]:
df.to_csv('aau_members-geocoded.csv')

In [40]:
print(df.latitude.isnull().sum())
print(df.longitude.isnull().sum())

38
38


In [35]:
df = df[pd.notnull(df['latitude'])]
print(df.latitude.isnull().sum())
print(df.longitude.isnull().sum())

0
0


In [36]:
from shapely.geometry import Point
points = gpd.GeoSeries(df["geometry"][0]) # todo: fix this
centroid = points.centroid
print(points.centroid)

0    POINT (-88.27755 40.03948)
dtype: geometry


In [38]:
import folium
import json

m = folium.Map(
    location=[points.centroid.y, points.centroid.x],
    zoom_start=3,
    tiles="Stamen Toner"
)

ala = folium.FeatureGroup(name="ALA Accredited Institutions")
for idx, row in df.iterrows():
    # folium.Marker(location=[row["latitude"], row["longitude"]], tooltip=row["name"]).add_to(m)
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]], 
        tooltip=row["institution"], 
        radius=10,
        color="#9C1944",
        fill=True,
        fill_color="#9C1944"
        ).add_to(m)

folium.LayerControl().add_to(m)
m.save('map.html')
m